In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
pip install langchain langchain-openai duckdb matplotlib pandas langgraph

In [ ]:
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from dotenv import load_dotenv

from langgraph.prebuilt import create_react_agent


In [ ]:
# CSV Paths
finance_file = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_finance_packaging.csv"
inventory_file = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_inventory_packaging.csv"
spend_file = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_spend_packaging.csv"
sales_file = "/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_sales_packaging.csv"

# Shared DuckDB connection
con = duckdb.connect(database=':memory:')

# Register views
con.execute(f"CREATE VIEW sales AS SELECT * FROM read_csv_auto('{sales_file}')")
con.execute(f"CREATE VIEW spend AS SELECT * FROM read_csv_auto('{spend_file}')")
con.execute(f"CREATE VIEW inventory AS SELECT * FROM read_csv_auto('{inventory_file}')")
con.execute(f"CREATE VIEW finance AS SELECT * FROM read_csv_auto('{finance_file}')")

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
sql_agent_prompt_prefix = """
You are a SQL expert agent following ReAct reasoning.

- You must ALWAYS output Thought -> Action -> Observation -> Final Answer.
- DO NOT output meta commentary.
- After seeing Observation results (SQL query output), ALWAYS extract concrete values.
- ALWAYS summarize the result table to answer the user’s original question directly.
- NEVER say "the query successfully identifies..." — always give actual values.
- DO NOT wrap SQL code in markdown formatting or backticks.
- ONLY output valid SQL without formatting.
- If column names contain spaces, enclose them in double quotes.
- The SQL dialect is SQLite (compatible with DuckDB).
- ALWAYS use the available tools (sql_db_query) to execute your queries.
- If the question involves visualization — also use viz_tool.
- NEVER just write SQL queries.
- ALWAYS call the action sql_db_query with the query as input.
- You are allowed to chain multiple queries to answer the question.
- If you encounter repeated errors or cannot execute the SQL query, still follow the ReAct format.
- When unable to answer, output:
Thought: I am unable to answer.
Final Answer: Unable to retrieve the data due to internal error.
- Do not write freeform explanations.
- Never write paragraphs describing failure.
- NEVER output markdown formatting.
- NEVER output queries inside triple backticks or code fences.
"""


In [ ]:
def run_sql_query(query: str) -> str:
    query = query.strip()
    if query.startswith("```sql"):
        query = query[6:].strip()
    if query.startswith("```"):
        query = query[3:].strip()
    if query.endswith("```"):
        query = query[:-3].strip()

    print("\n⚙️ Cleaned SQL:\n", query)

    df = con.execute(query).df()
    print("\n📊 SQL Result:\n", df.head())
    return df.to_string()

sql_tool = Tool(
    name="sql_db_query",
    func=run_sql_query,
    description="Executes SQL queries on sales, spend, inventory, finance data."
)

In [ ]:
def visualize_sql_result(query: str) -> str:
    query = query.strip()
    if query.startswith("```sql"):
        query = query[6:].strip()
    if query.startswith("```"):
        query = query[3:].strip()
    if query.endswith("```"):
        query = query[:-3].strip()

    print("\n⚙️ SQL for Visualization:\n", query)

    df = con.execute(query).df()
    print("\n📊 Data for Visualization:\n", df.head())

    if 'Date' in df.columns[0] or 'date' in df.columns[0] or 'month' in df.columns[0].lower():
        plt.figure(figsize=(10,6))
        plt.plot(df.iloc[:,0], df.iloc[:,1], marker='o')
    else:
        df.plot(kind='bar', x=df.columns[0], y=df.columns[1], figsize=(10,6))

    plt.title("Generated Chart")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("auto_chart.png")

    return "Chart saved as auto_chart.png"

viz_tool = Tool(
    name="viz_tool",
    func=visualize_sql_result,
    description="Generate charts from SQL query result."
)


In [ ]:
# Tools
tools = [sql_tool, viz_tool]

In [ ]:
from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    llm,
    tools,
    prompt=sql_agent_prompt_prefix
)


In [ ]:
response = graph.invoke({"input": "Plot monthly sales trend from the sales data"})
print(response)